In [18]:
# default_exp Web

In [19]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
#export
import os
import sys
import shutil
from jinja2 import Environment, FileSystemLoader, Template

# Web functions 

> API details.

In [21]:
#hide
from nbdev.showdoc import *

cPath = os.path.dirname(os.getcwd())
cPath
sys.path.insert(0,cPath)

In [22]:
#export
from refreshment.School import StudySystem
from refreshment.Program import Program, Subject, Record, Lesson
from refreshment.munge import makePriorsForSubject, removeAuthor, dateGrid,initials

In [23]:
#hide
guide = StudySystem.load("./data/programs.json")
for x in guide.programs:
    print(x.name)
    for y in x.subjects:
        print("\"" + x.name + "\",\"" + y.name + "\"")

Cathedral
"Cathedral","Reading"
"Cathedral","Music"
"Cathedral","Chapel"
"Cathedral","Science"
"Cathedral","HealthAndExercise"
"Cathedral","Schedules"
"Cathedral","Math"
"Cathedral","Mindful"
"Cathedral","EmotionalLearning"
"Cathedral","Writing"


In [24]:
#hide
cath = guide.programs[0]
foo = cath.subjects
sorted(foo, key=lambda x: x.name)[:2]

[Chapel; [CSB Lower School Distance Chapel March 31th, 2020|1|1|CSB Lower School Distance Chapel March 31th, 2020.mp4,CSB Lower School Distance Chapel April 7th, 2020|2|2|CSB Lower School Distance Chapel April 7th, 2020.mp4,CSB Lower School Distance Chapel April 21st, 2020|3|3|CSB Lower School Distance Chapel April 21st, 2020.mp4,CSB Lower School Distance Chapel April 28th, 2020|4|4|CSB Lower School Distance Chapel April 28th, 2020.mp4,CSB Lower School Distance Chapel May 5th, 2020|5|5|CSB Lower School Distance Chapel May 5th, 2020.mp4,Kindergarten Readiness - Lower School Chapel|6|6|Kindergarten Readiness - Lower School Chapel.mp4,CSB Lower School Distance Chapel May 12th, 2020|7|7|CSB Lower School Distance Chapel May 12th, 2020.mp4],
 EmotionalLearning; [Emotions|1|1|Emotions.mp4,Fillbucket|2|2|Fillbucket.mp4,Creative|3|3|Creative.mp4,Introducing Kimochis Bug|4|4|IntroducingKimochisBug.mp4,Gratitude|5|5|Gratitude.mp4,Gratitude 422|6|6|Gratitude 422.mp4,Gratitude.4.21|7|7|Gratitude.4.

In [25]:
#hide
template = Template('Hello {{ name }}!')
template.render(name=cath.name)

'Hello Cathedral!'

In [26]:
#export

class Render:
    def __init__(self, program, guide , dir="../../web",template = "../templates"):
       
        self.guide = guide
        self.program = [x for x in guide.programs if x.name == program][0]
        print("loaded program " + self.program.name)
        self.outputdir = os.path.join(dir,self.program.name)
        self.template = template
        
        self.file_loader = FileSystemLoader(self.template)
        self.env = Environment(loader=self.file_loader)
        self.resources = "resources"

        
    def basePath(self):
        return self.outputdir
    
    def renderLesson(self,sub,lesson):
        makePriorsForSubject(sub)
        template = self.env.get_template("lesson.html")
        resDir = "../../.."
        
        output = template.render(item=sub,
                                 resDir=resDir,
                                 less=lesson,
                                 program = self.program,
                                 styleDir=os.path.join("..",self.resources))
        subDir = os.path.join(self.outputdir,sub.name)
        
        if not os.path.exists(subDir):
            os.mkdir(subDir)
        f = open(os.path.join(subDir,"vid_" +str(lesson.key) + ".html"), "w")
        f.write(output)
        
    def renderSubject(self,sub):
        makePriorsForSubject(sub)
        template = self.env.get_template("subject.html")
        videos = [x for x in sub.lessons if x.fileName.endswith(".mp4")] 
        resources = [x for x in sub.lessons if not x.fileName.endswith(".mp4")]
        for x in sub.lessons:
            self.renderLesson(sub,x)
        resDir = "../../.."
        #print(sub.sequences)
        output = template.render(item=sub,
                                 videos=videos,
                                 res=resources,
                                 resDir=resDir,
                                 program = self.program,
                                 styleDir=os.path.join("..",self.resources))
        subDir = os.path.join(self.outputdir,sub.name)
        
        if not os.path.exists(subDir):
            os.mkdir(subDir)
        f = open(os.path.join(subDir,"index.html"), "w")
        f.write(output)
       
    def renderCalendar(self):
        
        grid = reversed(dateGrid(self.program))

        if not os.path.exists(self.outputdir):
            os.mkdir(self.outputdir)
  
        template = self.env.get_template("calendar.html")
        output = template.render(program=self.program,
                                 grid=grid,
                                 styleDir=os.path.join(".",self.resources))
        
        f = open(os.path.join(self.outputdir,"calendar.html"), "w")
        f.write(output)
        f.close()
        
    def renderSchool(self):
        
        foo = self.program.subjects
        seq = sorted(foo, key=lambda x: x.name)

        if not os.path.exists(self.outputdir):
            os.mkdir(self.outputdir)
            
        for x in seq:
            self.renderSubject(x)
            
        template = self.env.get_template("school.html")
        output = template.render(program=self.program,
                                 seq=seq,
                                 styleDir=os.path.join(".",self.resources))
        
        f = open(os.path.join(self.outputdir,"index.html"), "w")
        f.write(output)
        f.close()
        
        self.addResources()
        
    def addResources(self):
        resDir = os.path.join(self.outputdir,self.resources)
        if not os.path.exists(resDir):
            os.mkdir(resDir)
        startDir = os.path.join(".", self.resources)
        src_files = os.listdir(startDir)
        
        for file_name in src_files:
            full_file_name = os.path.join(startDir, file_name)
            if os.path.isfile(full_file_name):
                shutil.copy(full_file_name, os.path.join(resDir, file_name))
       
    def addFiles(self):
        for sub in self.program.subjects:
            for lesson in sub.lessons:
                source = os.path.join(self.guide.origin, self.program.name,sub.name,lesson.fileName)
                dest = os.path.join(self.outputdir,sub.name,lesson.fileName)
                
                if not os.path.isfile(dest):
                    print("found " + dest)
                    shutil.copy(source, dest)
        


In [27]:
#hide
guide.loadDirectory()
cath = guide.programs[0]
removeAuthor(guide)
baseRender = Render("Cathedral",guide)
baseRender.renderSchool()
#style = "background-color: #ABAD9E;"

loaded program Cathedral


In [28]:
#hide
baseRender = Render("Cathedral",guide)
baseRender.addFiles()

loaded program Cathedral
found ../../web/Cathedral/Writing/WritingAReview.mp4


In [29]:
baseRender.renderCalendar()

In [30]:
import json
#print(json.dumps(guide.toDict(), sort_keys=True, indent=4)) 

In [31]:
guide.origin

'../..'